In [1]:
import tensorflow as tf
import keras_tuner
import numpy as np


import pandas as pd
import openpyxl # Needed for reading excel
import pathlib

import decomposition
import models
import data
from metrics import smape
from windower import WindowGenerator
import hp_training
import results

2023-03-10 11:38:45.989448: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
cwd = pathlib.Path.cwd()

code_directory = cwd.parents[1]
gonem_directory = code_directory / "notebooks" / "Gonem"
model_directory = gonem_directory / 'models'

model = 'SS'
product = 'maize'
model_path =  model_directory / f"{model}_{product}"
checkpoint_path = model_path / f"{model}_{product}"


In [3]:
df = data.get_data(directory_path=gonem_directory, product=product)
df = df.iloc[:-2]
df.describe()

AVG_TAVG                                                  \
PARTNER_Labels      Brazil      France     Germany     Hungary     Ukraine   
count           214.000000  214.000000  214.000000  214.000000  214.000000   
mean            238.152301  125.494113  104.143755  116.576205   96.484903   
std              24.100760   55.456797   65.178828   79.286625   90.200095   
min             146.692308   20.168095  -29.860742  -50.419892  -98.247057   
25%             227.991548   77.127932   48.280935   44.350331   16.710027   
50%             245.982781  121.548459  102.466712  122.246111   92.398763   
75%             254.746595  177.640994  163.845330  190.244695  183.040152   
max             276.134483  231.747995  229.864177  242.571429  239.800437   

               Corn Price Futures    MAX_TMAX                          \
PARTNER_Labels             Global      Brazil      France     Germany   
count                  214.000000  214.000000  214.000000  214.000000   
mean                   449.422897  345.918892  225.531663  218.804582   
std                    151.626084   24.273653   67.153277   81.278888   
min                    201.750000  282.000000  107.622222   36.388889   
25%                    356.500000  331.000000  163.656487  140.181174   
50%                    390.125000  345.000000  228.556851  226.650735   
75%                    562.312500  360.425000  284.822581  287.334967   
max                    818.250000  401.000000  362.382979  371.823529   

                            ... renewable_energy_consumption_perc_of_total  \
PARTNER_Labels     Hungary  ...                                     Brazil   
count           214.000000  ...                                 214.000000   
mean            238.560202  ...                                  45.976355   
std              85.984321  ...                                   1.906558   
min              43.666667  ...                                  41.710000   
25%             163.500000  ...                                  44.876250   
50%             243.400000  ...                                  46.697083   
75%             316.700000  ...                                  47.570000   
max             392.500000  ...                                  48.920000   

                                                                \
PARTNER_Labels      France     Germany     Hungary     Ukraine   
count           214.000000  214.000000  214.000000  214.000000   
mean             12.883762   13.639136   13.302453    4.538575   
std               2.273139    2.851412    2.688740    2.132976   
min               8.520000    7.280000    7.290000    1.270000   
25%              11.155000   11.109375   12.801875    2.810000   
50%              13.286667   13.971250   13.640000    3.498750   
75%              15.310000   16.448125   15.377917    7.082500   
max              15.530000   17.170000   17.180000    7.440000   

               unemployment_total                                      \
PARTNER_Labels             Brazil      France     Germany     Hungary   
count                  214.000000  214.000000  214.000000  214.000000   
mean                    10.125584    8.969860    5.587944    6.981075   
std                      2.288750    0.867082    2.216610    2.708968   
min                      6.760000    7.390000    3.140000    3.420000   
25%                      8.205833    8.086250    3.646250    4.070833   
50%                      9.536250    9.068750    5.011250    7.343333   
75%                     12.473750    9.789167    7.403125    9.545000   
max                     13.700000   10.350000   11.170000   11.170000   

                            
PARTNER_Labels     Ukraine  
count           214.000000  
mean              8.363902  
std               1.103329  
min               6.350000  
25%               7.477500  
50%               8.522083  
75%               9.293125  
max               9.830000  

[8 rows x 63 columns]

In [4]:
label_columns = ['price']
label_columns = df.columns[df.columns.get_level_values(0).isin(label_columns)].tolist()
label_columns

[('price', 'Brazil'),
 ('price', 'France'),
 ('price', 'Germany'),
 ('price', 'Global'),
 ('price', 'Hungary'),
 ('price', 'Ukraine')]

In [5]:
stl = decomposition.STLDecomposer(labels=label_columns, period=12)
log = decomposition.Logger(labels=label_columns)
std = decomposition.Standardizer()

preproc = decomposition.Processor().add(stl).add(log).add(std)

In [6]:
width = 24
label_width = 6
shift = 6

window = WindowGenerator(input_width=width, label_width=label_width, shift=shift, data=df, 
                    # train_begin=0, train_end=.9, val_begin=None, val_end=.96,
                    train_begin=0, train_end=.97, val_begin=None, val_end=None,
                    # train_begin=0, train_end=.5, val_begin=None, val_end=.8,
                    test_begin=None, test_end=1., connect=True, remove_labels=True, label_columns=label_columns)
window.preprocess(preproc)
window

Total window size: 30
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29]
Label column name(s): [('price', 'Brazil'), ('price', 'France'), ('price', 'Germany'), ('price', 'Global'), ('price', 'Hungary'), ('price', 'Ukraine')]

In [7]:
label_std = decomposition.Standardizer(mean=std.mean[window.label_columns], std=std.std[window.label_columns])
label_log = decomposition.Logger(label_indices=range(len(window.label_columns)))
postproc = decomposition.Processor().add(label_std).add(label_log)
window.add_label_postprocess(postproc)

In [8]:
tuner = hp_training.get_tuner(model, gonem_directory/'hp', window)

2023-03-10 11:38:49.764976: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-10 11:38:49.769445: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-10 11:38:49.769780: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-10 11:38:49.770864: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
INFO:tensorflow:Reloading Tuner from /code/notebooks/Gonem/hp/SS/tuner0.json


In [9]:
# hp_training.run(tuner, window)

In [10]:
best_hps = tuner.get_best_hyperparameters()[0]
best_hps.values

{'lstm_units': 416,
 'lstm_layers': 0,
 'dense_units': 416,
 'dense_layers': 0,
 'heads': 2,
 'dropout': 0.38972766578190643,
 'key_dim': 128,
 'l1': 0.0009215140300448076,
 'l2': 0.0002556079498371441,
 'learning_rate': 0.0012848169081952467,
 'tuner/epochs': 200,
 'tuner/initial_epoch': 67,
 'tuner/bracket': 3,
 'tuner/round': 3,
 'tuner/trial_id': '0203'}

In [11]:
hp_training.final_train(tuner, window, checkpoint_path)

Epoch 1/200


2023-03-10 11:38:59.047423: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-10 11:39:01.522140: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-10 11:39:01.866029: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fee80048e40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-10 11:39:01.866070: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-03-10 11:39:01.898313: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-10 11:39:02.276938: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the l

6/6 - 11s - loss: 157.1254 - mae: 0.7837 - mse: 1.0568 - mape: 347.0849 - smape: 152.3220 - val_loss: 141.8907 - val_mae: 0.9841 - val_mse: 1.6261 - val_mape: 684.7963 - val_smape: 137.1299 - 11s/epoch - 2s/step
Epoch 2/200
6/6 - 0s - loss: 141.5849 - mae: 1.1230 - mse: 2.0402 - mape: 953.1991 - smape: 136.2771 - val_loss: 138.1488 - val_mae: 1.0942 - val_mse: 1.9190 - val_mape: 965.2861 - val_smape: 132.7905 - 388ms/epoch - 65ms/step
Epoch 3/200
6/6 - 0s - loss: 138.5153 - mae: 1.0946 - mse: 1.9477 - mape: 893.2922 - smape: 133.0176 - val_loss: 131.1284 - val_mae: 0.9213 - val_mse: 1.4654 - val_mape: 762.3817 - val_smape: 125.7709 - 382ms/epoch - 64ms/step
Epoch 4/200
6/6 - 0s - loss: 128.5859 - mae: 0.8831 - mse: 1.3688 - mape: 774.8044 - smape: 122.6349 - val_loss: 121.3675 - val_mae: 0.7601 - val_mse: 1.0749 - val_mape: 623.0065 - val_smape: 115.7356 - 383ms/epoch - 64ms/step
Epoch 5/200
6/6 - 0s - loss: 123.0312 - mae: 0.7845 - mse: 1.1217 - mape: 729.3643 - smape: 117.5150 - val_

In [12]:
m = tuner.hypermodel.build(best_hps)
m.load_weights(checkpoint_path)
m.evaluate(window.test)

1/1 [==============================] - 1s 964ms/step - loss: 53.3032 - mae: 0.9734 - mse: 2.0474 - mape: 78.9494 - smape: 49.2516


[53.30322265625,
 0.9734498858451843,
 2.047431230545044,
 78.94940185546875,
 49.251590728759766]

In [13]:
# w.test

# # val_performance['1'] = m.evaluate(w.val)
# for i in range(6):

#     label = label_columns[i]
#     print(label)
#     # performance['1'] = m.evaluate(w.test)
#     w.plot(m, plot_col=label, max_subplots=7)


In [14]:
inputs, labels, predictions, weights, mcds = [], [], [], [], []
for x, y in window.test.take(40):
    inputs.append(x)
    lab = y
    lab = window.label_postprocessor.reverse(lab)
    labels.append(lab)
    
    pred = m(x)
    pred = window.label_postprocessor.reverse(pred)
    predictions.append(pred)
    
    weight = m.attention_layer(x, return_weights=True)[1]
    weights.append(weight)
    
    mcd = results.monte_carlo_dropout(x, m, 100, window.label_postprocessor.reverse)
    mcds.append(mcd)
    
inputs = tf.concat(inputs, axis=0)
labels = tf.concat(labels, axis=0)
weights = tf.concat(weights, axis=0)
weights = tf.reduce_mean(weights, axis=0)
predictions = tf.concat(predictions, axis=0)
mcds = tf.concat(mcds, axis=1)

In [15]:
np.save(model_path / f"{product}_inputs", inputs.numpy())
np.save(model_path / f"{product}_labels", labels.numpy())
np.save(model_path / f"{product}_weights", weights.numpy())
np.save(model_path / f"{product}_predictions", predictions.numpy())
np.save(model_path / f"{product}_mcd_predictions", mcds.numpy())
